In [ ]:
import json
import httpx
import dotenv
import os

dotenv.load_dotenv("../config/.env.development")


**Fetching Access Token**

In [ ]:
url = os.environ["SERVER_URL"] + "/authentication"
body = {
    "username": os.environ["SERVER_USERNAME"],
    "password": os.environ["SERVER_PASSWORD"]
}

auth_response = httpx.post(url=url, json=body)
assert str(auth_response.status_code).startswith("2"), auth_response
print(json.dumps(auth_response.json(), indent=4))
access_token = auth_response.json()['access_token']

**Creating a Sensor**

In [ ]:
url = os.environ["SERVER_URL"] + "/sensors"
headers = {
    "authorization": f"Bearer {access_token}"
}
body = {
    "sensor_name": "tum-esm-midcost-raspi-20",
    "network_identifier": "81bf7042-e20f-4a97-ac44-c15853e3618f",
    "configuration": {
        "version": "0.1.0-alpha.11",
        "verbose_logging": False,
        "active_components": {
            "calibration_procedures": False,
            "mqtt_communication": True,
            "heated_enclosure_communication": False,
            "pump_speed_monitoring": False
        },
        "hardware": {
            "pumped_litres_per_round": 0.003,
            "inner_tube_diameter_millimiters": 4
        },
        "measurement": {
            "timing": {
                "seconds_per_measurement_interval": 120,
                "seconds_per_measurement": 10
            },
            "pumped_litres_per_minute": 4,
            "air_inlets": [
                { "valve_number": 1, "direction": 300, "tube_length": 50 },
                { "valve_number": 2, "direction": 50, "tube_length": 50 }
            ]
        },
        "calibration": {
            "hours_between_calibrations": 25,
            "gases": [
                { "valve_number": 3, "concentration": 400 },
                { "valve_number": 4, "concentration": 800 }
            ],
            "flushing": {
                "seconds": 300,
                "pumped_litres_per_minute": 0.5
            },
            "sampling": {
                "pumped_litres_per_minute": 0.5,
                "sample_count": 20,
                "seconds_per_sample": 300
            },
            "cleaning": {
                "seconds": 300,
                "pumped_litres_per_minute": 0.5
            }
        },
        "heated_enclosure": {
            "target_temperature": 40,
            "allowed_deviation": 3
        }
    }
}

sensor_creation_response = httpx.post(url=url, headers=headers, json=body)
assert str(sensor_creation_response.status_code).startswith("2"), sensor_creation_response
print(json.dumps(sensor_creation_response.json(), indent=4))

**Store list of sensor identifiers**

In [ ]:
MQTT_SENSOR_IDENTIFIERS = {
    "moritz-macbook-pro-14": "e04a6cb9-88c5-43b4-887d-72615d15d1e3",
    "tum-esm-midcost-raspi-1": "c04e0bcc-2b32-4fb3-8971-9cbe27ab7117",
    "tum-esm-midcost-raspi-2": "64c5c8ec-4e6b-413b-b113-b130f80eae91",
    "tum-esm-midcost-raspi-3": "3682334d-a359-438c-ad40-860270bbcbf0",
    "tum-esm-midcost-raspi-4": "919ac396-de7d-4dda-8224-564739e0ff1b",
    "tum-esm-midcost-raspi-5": "df2727fd-0f22-4c39-bc46-27a3c632087a",
    "tum-esm-midcost-raspi-6": "870093ad-5773-458a-a7a9-73fb7c66d2e9",
    "tum-esm-midcost-raspi-7": "7d0747e7-7c2d-4d2e-bd19-e98ae29d5948",
    "tum-esm-midcost-raspi-8": "58ae94f3-e2d3-4e16-b3c4-9daf31648c6b",
    "tum-esm-midcost-raspi-9": "7d2ba05f-4233-4a2f-b00b-452d9a34ee18",
    "tum-esm-midcost-raspi-10": "fcce393d-ac53-4b59-ae71-d89cc2cdd619",
    "tum-esm-midcost-raspi-11": "ecc83b80-bf8a-4f70-b56f-0e0a5d071f9d",
    "tum-esm-midcost-raspi-12": "2794eda8-216f-4ac7-aea9-68734dcbb5ac",
    "tum-esm-midcost-raspi-13": "5af58695-ae8d-419a-8f2a-1ae2017b9913",
    "tum-esm-midcost-raspi-14": "07195901-387e-4218-b8c7-e811c247b94b",
    "tum-esm-midcost-raspi-15": "9573732f-e183-4097-89d6-a353e36dba69",
    "tum-esm-midcost-raspi-16": "8cd4e335-e200-4a69-b5e9-01bb84fc960a",
    "tum-esm-midcost-raspi-17": "f695a691-6ab8-4b30-bee0-6c4aa7e07f8f",
    "tum-esm-midcost-raspi-18": "c7e8c239-0f86-45e2-b83d-c70accb2ece5",
    "tum-esm-midcost-raspi-19": "c22a5292-3020-4e31-9920-8053155da1e6",
    "tum-esm-midcost-raspi-20": "bcbf556b-e3b1-452b-b92c-8562d874e328",
}

**Sending data**

In [ ]:
import sys
import time
import dotenv

sys.path.append("..")
dotenv.load_dotenv("../config/.env")
dotenv.load_dotenv("../config/.env.development")

from src import custom_types, utils

utils.SendingMQTTClient.init_archiving_loop_process()
utils.SendingMQTTClient.init_sending_loop_process()
utils.SendingMQTTClient.check_errors()

config = utils.ConfigInterface.read()
sending_mqtt_client = utils.SendingMQTTClient()
sending_mqtt_client.enqueue_message(
    config,
    custom_types.MQTTLogMessageBody(
        severity="info",
        revision=config.revision,
        timestamp=time.time(),
        subject="Hello, world!",
        details="no one will read this anyways"
    )
)
sending_mqtt_client.enqueue_message(
    config,
    custom_types.MQTTDataMessageBody(
        revision=config.revision,
        timestamp=time.time(),
        value=custom_types.MQTTCO2Data(
            variant="co2",
            data=custom_types.CO2SensorData(
                raw=400,
                compensated=500,
                filtered=600
            )
        )
    )
)

time.sleep(10)
utils.SendingMQTTClient.check_errors()


**Show active MQTT queue**

In [ ]:
import sys
import time
import dotenv

sys.path.append("..")
dotenv.load_dotenv("../config/.env")
dotenv.load_dotenv("../config/.env.development")

from src import custom_types, utils

active_queue = utils.ActiveMQTTQueue()
active_queue.get_rows_by_status("in-progress")

**Update the Config**

In [ ]:

new_config = {
    "version": "0.1.0-beta.2",
    "verbose_logging": False,
    "active_components": {
        "calibration_procedures": True,
        "mqtt_communication": True,
        "heated_enclosure_communication": True,
        "pump_speed_monitoring": False
    },
    "hardware": {
        "pumped_litres_per_round": 0.000833,
        "inner_tube_diameter_millimiters": 4
    },
    "measurement": {
        "timing": {
            "seconds_per_measurement_interval": 120,
            "seconds_per_measurement": 600
        },
        "pumped_litres_per_minute": 0.5,
        "air_inlets": [
            { "valve_number": 1, "direction": 0, "tube_length": 50 }
        ]
    },
    "calibration": {
        "start_timestamp": 1682503200,
        "hours_between_calibrations": 25,
        "seconds_per_gas_bottle": 600,
        "gases": [
            { "valve_number": 2, "concentration": 400 },
            { "valve_number": 3, "concentration": 800 }
        ]
    },
    "heated_enclosure": {
        "target_temperature": 25,
        "allowed_deviation": 3,
        "seconds_per_stored_datapoint": 60
    }
}



In [ ]:
for raspi_number in range(1, 2):
    print(f"raspi {raspi_number}")
    url = os.environ["SERVER_URL"] + "/sensors/" + MQTT_SENSOR_IDENTIFIERS[f'tum-esm-midcost-raspi-{raspi_number}']
    headers = {
        "authorization": f"Bearer {access_token}"
    }
    body = {
        "sensor_name": f'tum-esm-midcost-raspi-{raspi_number}',
        "network_identifier": "81bf7042-e20f-4a97-ac44-c15853e3618f",
        "configuration": new_config,
    }

    config_update_response = httpx.put( url=url, headers=headers, json=body)
    assert str(config_update_response.status_code).startswith("2"), config_update_response.text
    print(json.dumps(config_update_response.json(), indent=4))

**Receive Config Messages**

In [ ]:
import sys
import time
import dotenv

dotenv.load_dotenv("../config/.env")

sys.path.append("..")
from src import custom_types, utils, procedures

config = utils.ConfigInterface.read()
logger = utils.Logger("notebook")
logger.info("hello")
procedures.MessagingAgent.init(config)
procedures.MessagingAgent.check_errors()

for i in range(4):
    print(procedures.MessagingAgent.get_config_message())
    time.sleep(1)

procedures.MessagingAgent.check_errors()